<a href="https://colab.research.google.com/github/wandb/edu/blob/main/decision-opt-course/2_bimbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{decisionopt-nb2b} -->

# Lesson 2 - Applying decision optimization for regression

In [ ]:
%pip install wandb -Uqqq

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path
import os
import wandb

os.environ["WANDB_QUIET"] = "true"  # Keep notebook output clean
wandb_project = "decision_opt_bimbo"
plt.style.use("fivethirtyeight")

# Let's load the data from a W&B artifact
with wandb.init(project=wandb_project) as run:
    artifact = run.use_artifact(
        "wandb_course/decision_opt/grupo-bimbo-inventory-demand:latest"
    )
    data_dir = Path(artifact.download())

data = pd.read_csv(data_dir / "train.csv")
clientes = pd.read_csv(data_dir / "cliente_tabla.csv")
productos = pd.read_csv(data_dir / "producto_tabla.csv")
town_state = pd.read_csv(data_dir / "town_state.csv")

# Merge datasets
data = data.merge(clientes, on="Cliente_ID", how="left")
data = data.merge(productos, on="Producto_ID", how="left")
data = data.merge(town_state, on="Agencia_ID", how="left")

data.head()

In [ ]:
# Define the categorical columns
categorical_cols = ["Agencia_ID", "Canal_ID", "Ruta_SAK", "Cliente_ID", "Producto_ID"]

# Define the label encoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])
    label_encoders[col] = le

num_unique_vals = {col: data[col].nunique() for col in categorical_cols}
embedding_sizes = {col: min(50, num_unique_vals[col] // 2) for col in categorical_cols}

# Split into features and target
X = data[categorical_cols].values
y = data["Demanda_uni_equil"].values

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


# Define the Dataset class
class BimboDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(X[:, i], dtype=torch.long) for i in range(X.shape[1])]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [x[idx] for x in self.X], self.y[idx]


# Create Datasets and DataLoaders
train_dataset = BimboDataset(X_train, y_train)
val_dataset = BimboDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)


# Define the model
class SimpleModel(nn.Module):
    def __init__(self, embedding_sizes, hidden_size=128):
        super(SimpleModel, self).__init__()
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(num_unique_vals[col], embedding_sizes[col])
                for col in categorical_cols
            ])
        self.fc1 = nn.Linear(sum(embedding_sizes.values()), hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = [embedding(x_i) for x_i, embedding in zip(x, self.embeddings)]
        x = torch.cat(x, dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x).squeeze(-1)
        return x


def train_model(loss_fn, num_epochs=5):
    model = SimpleModel(embedding_sizes)
    optimizer = optim.Adam(model.parameters(), lr=0.005)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = loss_fn(outputs, targets)
                val_loss += loss.item()
                val_preds.extend(outputs.tolist())
                val_targets.extend(targets.tolist())

        val_loss /= len(val_loader)
        r2 = r2_score(val_targets, val_preds)
        wandb.log(
            {
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "r_squared": r2,
            }
)
    return model, np.array(val_preds), np.array(val_targets)

In [ ]:
def log_business_metrics(stocking_decisions, actual_demand, name, tags):
    with wandb.init(
        project=wandb_project,
        name=name,
        tags=tags,
        job_type="decision",
    ):
        frac_understocks = (stocking_decisions < actual_demand).mean()
        total_understocked_amt = (actual_demand - stocking_decisions).clip(0).sum()
        frac_overstocks = (stocking_decisions > actual_demand).mean()
        total_overstocked_amt = (stocking_decisions - actual_demand).clip(0).sum()
        utility = -3 * total_understocked_amt - total_overstocked_amt
        mae = mean_absolute_error(actual_demand, stocking_decisions)
        mse = mean_squared_error(actual_demand, stocking_decisions)
        r2_score(actual_demand, stocking_decisions),

        wandb.log(
            {
                "frac_understocks": frac_understocks,
                "total_understocked_amt": total_understocked_amt,
                "frac_overstocks": frac_overstocks,
                "total_overstocked_amt": total_overstocked_amt,
                "utility": utility,
                "mae": mae,
                "mse": mse,
                "r2_score": r2_score,
            }
        )
        return

In [ ]:

with wandb.init(
    project=wandb_project, name="mse_optimized", tags=["mse_loss"]
):
    loss = nn.MSELoss()
    mse_model, mse_val_preds, mse_val_targets = train_model(loss, num_epochs=5)
    # save mse_model as artifact
    torch.save(mse_model.state_dict(), "mse_model.pt")
    wandb.save("mse_model.pt")

mse_val_stock = np.ceil(mse_val_preds)
log_business_metrics(mse_val_stock, mse_val_targets, "mse_loss_predictions", tags=["mse_loss", "stock_predicted_sales"])


In [ ]:
alternative_stocking_rule = np.ceil(1.5 * mse_val_preds)
log_business_metrics(alternative_stocking_rule,
                     mse_val_targets,
                     "50_pct_above_mse_loss_predictions",
                     tags=["mse_loss", "stock_50_pct_above_predicted_sales"]
                     )

In [ ]:
with wandb.init(
    project=wandb_project, name="mae_optimized", tags=["mae_loss"]
):
    loss = nn.L1Loss()
    mae_model, mae_val_preds, mae_val_targets = train_model(loss, num_epochs=5)

mae_val_stock = np.ceil(mae_val_preds)
log_business_metrics(mae_val_stock,
                    mae_val_targets,
                    'mae_loss_predictions',
                    tags=["mae_loss", "stock_predicted_sales"]
                    )

In [ ]:
above_mae_stocking_rule = np.ceil(1.5 * mae_val_preds)
log_business_metrics(above_mae_stocking_rule,
                     mse_val_targets,
                     "50_pct_above_mae_loss_predictions",
                     tags=["mse_loss", "stock_50_pct_above_predicted_sales"]
                     )


Return to example of understock costing \\$3 per unit and overstock costing \\$1 per unit

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, outputs, actual):
        diff = outputs - actual
        loss = torch.where(outputs > actual, diff, -3 * diff)
        return loss.mean()


with wandb.init(
    project="regression_decision_opt", name="our_utility_loss", tags=["custom_loss"]
):
    custom_model, custom_val_preds, custom_val_targets = train_model(
        CustomLoss(), num_epochs=5
    )

custom_val_stock = np.ceil(custom_val_preds)
log_business_metrics(custom_val_stock, custom_val_targets, 'utility_fn_loss_predictions', tags=['stock_predicted_sales'])

## Predicting Full Distributions

Using QuantileRegressionForest: https://scikit-garden.github.io/examples/QuantileRegressionForests/

In [ ]:
%pip install quantile-forest -qqq

In [ ]:
from quantile_forest import RandomForestQuantileRegressor

qrf = RandomForestQuantileRegressor(
    n_estimators=100, min_samples_leaf=50, random_state=0
)
qrf.fit(X_train, y_train)

In [ ]:
quantiles = [i / 100 for i in range(5, 100, 5)]
sample_preds = qrf.predict(X_val, quantiles=quantiles)

In [ ]:
one_demand_prediction = sample_preds[5]
one_demand_prediction

In [ ]:
import math

def rarely_run_out_rule(prediction):
    outlier_bound = 3 * np.mean(prediction)
    to_stock = math.ceil(min(prediction[-2], outlier_bound))
    return to_stock

rarely_run_out_rule(one_demand_prediction)

In [ ]:
all_stocking_decisions = np.apply_along_axis(rarely_run_out_rule, 1, sample_preds)
(all_stocking_decisions < sample_preds[:, -2]).mean()

In [ ]:
log_business_metrics(all_stocking_decisions, y_val, 'capped_90th_percentile', tags=['probabilistic_forecast'])

In [ ]:
np.where(all_stocking_decisions < sample_preds[:, -2])

In [ ]:
print(f"stocked: {all_stocking_decisions[126]} when demand was {sample_preds[126, :]}")

In [ ]:
for row in [0, 126, 295, 298, 557, 620, 882]:
    print(f"stocked: {all_stocking_decisions[row]}. Input {sample_preds[row, :]}")